# Reconstruct phantom data
This exercise shows how to handle data from the Siemens mMR. It shows how to get from listmode data to sinograms, get a randoms estimate, and reconstruct using normalisation, randoms, attenuation and scatter.

It is recommended you complete the [OSEM_reconstruction notebook](OSEM_reconstruction.ipynb) first. Even better would be to look at the OSEM part of the [ML_reconstruct notebook](ML_reconstruct.ipynb) as well.

This exercise uses data from a phantom acquisition at UCL on a Siemens mMR. The phantom is the NEMA phantom (essentially a torso-shaped perspex box, with some spherical inserts). You will need to download that data. Please use the read INSTALL.md or DocForParticipants.md for details. 

The script should work for other data of course, but you will need to adapt filenames.

You can also adjust it to use other reconstruction algorithms than OSEM with very little changes.

Note that we currently don't show here how to extract the data from the console. Please
[check our wiki for more information](https://github.com/CCPPETMR/SIRF/wiki/PET-raw-data).

Authors: Kris Thielemans and Evgueni Ovtchinnikov  
First version: 8th of September 2016  
vSecond Version: 17th of May 2018-  
Third version: June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).  
Copyright 2015 - 2017, 2021 Rutherford Appleton Laboratory STFC.  
Copyright 2015 - 2018, 2020-2021 University College London.

This is software developed for the Collaborative Computational
Project in Synergistic Reconstruction for Biomedical Imaging
(http://www.ccpsynerbi.ac.uk/).

SPDX-License-Identifier: Apache-2.0

# Initial set-up

In [ ]:
#%% make sure figures appears inline and animations works
%matplotlib widget

Run `download_data.sh -p` if you didn't yet!

In [ ]:
%%bash
bash ../../scripts/download_data.sh -p

In [ ]:
# Setup the working directory for the notebook
import notebook_setup
from sirf_exercises import cd_to_working_dir
cd_to_working_dir('PET', 'reconstruct_measured_data')

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from sirf.Utilities import show_2D_array, examples_data_path
from sirf.STIR import *
from sirf_exercises import exercises_data_path

# Find the path to the data directory
data_path = exercises_data_path('PET', 'mMR', 'NEMA_IQ')

In [ ]:
# convert the Siemens Normalisation into STIR format, some keys slightly differ
# $data_path/20170809_NEMA[_MUMAP]_UCL.n.hdr are the extracted headers from the scanner
# norm.n.hdr and umap.v.hdr are the output converted files
#!convertSiemensInterfileToSTIR.sh $data_path/20170809_NEMA_UCL.n.hdr norm.n.hdr
!cp $data_path/20170809_NEMA_UCL.n.hdr norm.n.hdr
!convertSiemensInterfileToSTIR.sh $data_path/20170809_NEMA_MUMAP_UCL.v.hdr umap.v.hdr

# These files will sometimes have lines terminated with CR instead of CRLF, fix this
# This just means, if we see CR<character> and <character> isn't LF, replace it with CRLF<character>
!sed -i.bak "s/\r\([^\n]\)/\r\n\1/g" norm.n.hdr
!sed -i.bak "s/\r\([^\n]\)/\r\n\1/g" umap.v.hdr

# Now add absolute data path to the header file
# This command prepends the data path to the data file so that the header in our working folder points to the data
# You won't need to do this for your own data if the data file is in the same directory.
!sed -i.bak2 -e "s#\(!name of data file:=\)#\\1{data_path}/#" umap.v.hdr
!sed -i.bak2 -e "s#\(!name of data file:=\)#\\1{data_path}/#" norm.n.hdr

# Advanced: if you'd like to have a look at what changed in the umap, uncomment below
# Lines starting with < are the original Siemens
# and lines starting with > are the STIR converted
# !diff $data_path/20170809_NEMA_MUMAP_UCL.v.hdr umap.v.hdr

If the above failed, it probably means that `convertSiemensInterfileToSTIR.sh` wasn't installed. The following line might help.

In [ ]:
# !cp $SIRF_PATH/../STIR/scripts/IO/convertSiemensInterfileToSTIR.sh $SIRF_INSTALL_PATH/bin

In [ ]:
#%% set filenames 
# input files
list_file = os.path.join(data_path, '20170809_NEMA_60min_UCL.l.hdr')
norm_file = 'norm.n.hdr'
attn_file = 'umap.v.hdr'
# output filename prefixes
sino_file = 'sino'
%ls

In [ ]:
# redirect STIR messages to some files
# you can check these if things go wrong
msg_red = MessageRedirector('info.txt', 'warnings.txt')

# Creating sinograms from listmode data
Modern PET scanners can store data in listmode format. This is essentially a long list of all events detected by the scanner. We are interested here in the *prompts* (the coincidence events) and the *delayed events* (which form an estimate of the *accidental coincidences* in the prompts.

We show how to histogram the prompts into a sinogram etc.

## First create a template for the sinogram
This template is used to specify the sizes of the output sinogram.

It is often the case in PET that we use sinograms with "larger" bins, i.e. combine data from several detector pairs into a single bin. This reduces size of the final sinogram, and decreases computation time. The terminology here is somewhat complicated, but *span* uses "axial compression" (higher span means smaller data size), *max_ring_diff* specifies the maximum ring difference to store, and *view_mash_factor* can be used to reduce the number of views (or azimutal angles). You could check the [STIR glossary](http://stir.sourceforge.net/documentation/STIR-glossary.pdf) for more detail.

Siemens uses span=11, max_ring_diff=60 and view_mash_factor=1 for the mMR. Here we will use a smaller data size to reduce computation time for the exercise. Feel free to change these numbers (if you know what you are doing...). (Note that the list mode data stores data only up to ring difference 60, even though the scanner has 64 rings).

In [ ]:
template_acq_data = AcquisitionData('Siemens_mMR', span=11, max_ring_diff=15, view_mash_factor=2)
template_acq_data.write('template.hs')

In [ ]:
# create listmode-to-sinograms converter object
lm2sino = ListmodeToSinograms()

# set input, output and template files
lm2sino.set_input(list_file)
lm2sino.set_output_prefix(sino_file)
lm2sino.set_template('template.hs')

In [ ]:
# set timing interval (in secs) since start of acquisition
# (the listmode file provided is for 1 hour).
# you can vary this to see the effect on noise. Increasing it will mean somewhat longer
# processing time in the following steps (but not in the reconstruction).
lm2sino.set_time_interval(0, 600)  # 0 - 600 is the first 10 minutes
# set up the converter
lm2sino.set_up()
# create the prompts sinogram
lm2sino.process()

In [ ]:
# check the content of the directory. there should be a `sino*.hs`, `'.s` pair.
# The `.hs` file is an Interfile header pointing to the binary data.
%ls

## Check the prompts sinograms
The 3D PET data returned by `as_array` are organised by 2D sinogram. The exact order of the sinograms
is complicated for 3D PET, but SIRF orders them by *segment* (roughly: average ring difference). The first
segment corresponds to "segment 0", i.e. detector pairs which are (roughly) in the same 
detector ring. For a clinical scanner with `N` rings, there will be `2N-1` (2D) sinograms in segment 0. See also information in the [image_creation_and_simulation notebook](image_creation_and_simulation.ipynb).

In [ ]:
# get access to the sinograms
acq_data = lm2sino.get_output()
# copy the acquisition data into a Python array
acq_array = acq_data.as_array()[0,:,:,:]  # first index is for ToF, which we don't have here
# how many counts total?
print('num prompts: %d' % acq_array.sum())
# print the data sizes. 
print('acquisition data dimensions: %dx%dx%d' % acq_array.shape)
# use a slice number for display that is appropriate for the NEMA phantom
# showing a "middle" sinogram in segment 0.
z = 71
show_2D_array('Acquisition data', acq_array[z,:,:])

Let's also show a horizontal profile through this sinogram.

In [ ]:
plt.figure()
plt.plot(acq_array[z,0,:],'.')
plt.xlabel('radial distance')
plt.ylabel('counts')

Clearly, there seems to be lots of things "wrong" with this sinogram. In the next few sections, we will show how to incorporate various effects into the acquisition model. We will also reconstruct the data with these increasingly accurate models for the acquisition as illustration.

For simplicity, we will use OSEM and use only a few sub-iterations for speed.

## Reconstruct the prompt data without any extra modelling
The simplest model that you can use for PET data is line integrals. So let's do that here and see how well it does.

In SIRF, we currently have 2 options:
- the ray tracing matrix multiplication which we have used in other demos
- Alternatively, you could use the [parallelproj acquisition model](https://github.com/gschramm/parallelproj/) which is faster if you have SIRF/STIR with GPU capability. However, note that our wrapper of parallelproj currently is inefficient for a large number of subsets, so you might want to reduce the number of subsets below when using this choice.

In [ ]:
#  by ray tracing
acq_model = AcquisitionModelUsingRayTracingMatrix()
acq_model.set_num_tangential_LORs(10);

In [ ]:
# using parallelproj
# acq_model = AcquisitionModelUsingParallelproj()

In [ ]:
# define objective function to be maximized as
# Poisson logarithmic likelihood (with linear model for mean)
obj_fun = make_Poisson_loglikelihood(acq_data)
obj_fun.set_acquisition_model(acq_model)

In [ ]:
# create the reconstruction object
recon = OSMAPOSLReconstructor()
recon.set_objective_function(obj_fun)

# Choose a number of subsets.
# For the mMR, best performance requires to not use a multiple of 9 as there are gaps
# in the sinograms, resulting in unbalanced subsets (which isn't ideal for OSEM).
num_subsets = 21
# Feel free to increase these.
# (Clinical reconstructions use around 60 subiterations, e.g. 21 subsets, 3 full iterations)
num_subiterations = 12
recon.set_num_subsets(num_subsets)
recon.set_num_subiterations(num_subiterations)

In [ ]:
# create initial image estimate of dimensions and voxel sizes
# compatible with the scanner geometry (included in the AcquisitionData
# object acq_data) and initialize each voxel to 1.0
nxny = (127, 127)
initial_image = acq_data.create_uniform_image(1.0, nxny)

In [ ]:
image = initial_image
recon.set_up(image)
# set the initial image estimate
recon.set_current_estimate(image)
# reconstruct
recon.process()
# show reconstructed image
image_array = recon.get_output().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

Obviously this image doesn't look great. We have used an acquisition model that just uses line integrals. Clearly not good enough!

## Add detector sensitivity modelling
Probably the obvious feature of the sinogram above are the diagonal lines. There are due to 2 effects:
- Each crystal pair will have different detection efficiency.
- The Siemens mMR (and other scanners) has some "gaps" between detector blocks, which are accomodated in the sinograms by creating a "virtual" crystal between each block. These will never detect any counts.

We need to take this into account
in our acquisition model. The scanner provides a *normalisation file* to do this (the terminology
originates from the days that we were "normalising" by  dividing by the detected counts 
by the sensitivities. Note that this cannot be done with the "virtual" crystals as it would lead to 0/0).

In SIRF, you can incorporate this effect in the acquisition model by using an `AcquisitionSensitivityModel`.

In [ ]:
# create it from the supplied file
asm_norm = AcquisitionSensitivityModel(norm_file)

We can use this to create sinogram data with the detection efficiencies. Let's do that here and display.

In [ ]:
asm_norm.set_up(acq_data)
det_efficiencies=acq_data.get_uniform_copy(1)
asm_norm.unnormalise(det_efficiencies)

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(det_efficiencies.as_array()[0,z,:,:],clim=None)
plt.subplot(1,2,2)
plt.plot(det_efficiencies.as_array()[0,z,0,:],'.')
plt.xlabel('radial distance')
plt.ylabel('efficiency')
plt.show()

Note that the scale of these detection efficiencies is a bit arbitrary. At this point in time (SIRF 3.4 with STIR 5.1), a global calibration factor is not yet taken into account for instance.

You can see the same diagonal patterns here. Therefore including this into our acquisition model will likely give much better results.

In [ ]:
# add it to the acquisition model
acq_model.set_acquisition_sensitivity(asm_norm)

In [ ]:
# update the objective function
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
# reconstruct
image = initial_image
recon.set_up(image)
recon.set_current_estimate(image)
recon.process()
# show reconstructed image
image_array = recon.get_output().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Add attenuation modeling

In [ ]:
# read attenuation image
attn_image = ImageData(attn_file)
z = 71
attn_image.show(z)

In [ ]:
attn_acq_model = AcquisitionModelUsingRayTracingMatrix()
asm_attn = AcquisitionSensitivityModel(attn_image, attn_acq_model)
# converting attenuation into attenuation factors (see previous exercise)
asm_attn.set_up(acq_data)
attn_factors = acq_data.get_uniform_copy(1)
print('applying attenuation (please wait, may take a while)...')
asm_attn.unnormalise(attn_factors)

In [ ]:
# use these in the final attenuation model
asm_attn = AcquisitionSensitivityModel(attn_factors)

Let's display the attenuation factors.

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(attn_factors.as_array()[0,z,:,:],clim=None)
plt.subplot(1,2,2)
plt.plot(attn_factors.as_array()[0,z,0,:],'.')
plt.xlabel('radial distance')
plt.ylabel('attenuation factor')
plt.show()

We now have two acquisition_sensitivity_models: for detection sensitivity and for
count loss due to attenuation. We combine them by "chaining" them together (which will
model the multiplication of both sensitivities).

In [ ]:
# chain attenuation and normalisation
asm = AcquisitionSensitivityModel(asm_norm, asm_attn)

In [ ]:
# update the acquisition model etc
acq_model.set_acquisition_sensitivity(asm)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
# reconstruct
image = initial_image
recon.set_up(image)
recon.set_current_estimate(image)
recon.process()
# show reconstructed image
image_array = recon.get_output().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Add a background term for modelling the randoms
PET data includes "accidental coincidences" (often called "randoms"). These occur when annihilation phantoms of 2 different annihilations are detected within the coincidence window. This gives a global background to the data. So we need to model this!

### Estimate the *randoms* background
Siemens stores *delayed coincidences*. These form a very noisy estimate of the
background due to accidental coincidences in the data. However, that estimate is too noisy
to be used in iterative image reconstruction. Note that the acquisition model should give you an estimate of the "mean" of the data (i.e. is noiseless).

SIRF uses an algorithm from STIR that gives a much less noisy estimate. The help message 
gives some information.

In [ ]:
help(lm2sino)

In [ ]:
# Get the randoms estimate
# This will take a while
randoms = lm2sino.estimate_randoms()

### Plot the randoms-estimate
A (2D) sinogram of the randoms has a similar pattern diagonal lines. This is related to the
detector efficiencies of course, but we cannot get into that here.

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(randoms.as_array()[0,z,:,:],clim=None)
plt.subplot(1,2,2)
plt.plot(randoms.as_array()[0,z,0,:],'.')
plt.xlabel('radial distance')
plt.ylabel('estimated randoms')
plt.show()

### Include the randoms estimate into the acquisition model

In [ ]:
acq_model.set_background_term(randoms)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
recon.set_current_estimate(initial_image)
recon.set_up(initial_image)
recon.process()
# show reconstructed image
image_array = recon.get_output().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

## Scatter
Finally, some of the detected counts will be from coincidences where one (or both) of the annihilation photons have scattered. Although the scanner tries to reject these by using energy windowing, there will still be a non-negligible fraction. This is due to the energy resolution of current PET scanners, and trade-offs made between detecting scattered counts and rejecting unscattered counts.

The most common way to estimate scatter in PET is "model-based". Essentially it is an iterative loop between image estimation and modelling the scatter based on the current image estimate (and the attenuation image). We can run the STIR implementation from inside SIRF as follows.

In [ ]:
se = ScatterEstimator()

se.set_input(acq_data)
se.set_attenuation_image(attn_image)
se.set_randoms(randoms)
se.set_asm(asm_norm)
# Unfortunately, the ScatterEstimator currently needs attenuation "correction" factors, which
# is what we need to multiply by to correct for attenuation, while we computed the attenuation
# factors above.
# Fortunately, these are simply the inverse.
acf_factors = attn_factors.get_uniform_copy()
acf_factors.fill(1/attn_factors.as_array())
# I could also have used the following (but it would take more time)
#asm_attn.normalise(acf_factors)
se.set_attenuation_correction_factors(acf_factors)

# set the number of iterations used for the scatter algorithm.
# The default is 5, but 3 is often enough, so we will use that here to reduce computation time.
se.set_num_iterations(3)
# optionally let it write intermediate scatter estimates to file
se.set_output_prefix('scatter_estimate')

In [ ]:
# go and compute it! (might take a minute or 2)
se.set_up()
se.process()
scatter_estimate = se.get_output()

Let's display it

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(scatter_estimate.as_array()[0,z,:,:],clim=None)
plt.subplot(1,2,2)
plt.plot(scatter_estimate.as_array()[0,z,0,:],'.')
plt.xlabel('radial distance')
plt.ylabel('estimated scatter')
plt.show()

You can see that the scatter estimate is a fairly smooth background, where the detection efficiencies are again superimposed.

## reconstruct including scatter and all other terms

In [ ]:
acq_model.set_background_term(randoms+scatter_estimate)
obj_fun.set_acquisition_model(acq_model)
recon.set_objective_function(obj_fun)

In [ ]:
recon.set_current_estimate(initial_image)
recon.process()
# show reconstructed image
image_array = recon.get_output().as_array()
show_2D_array('Reconstructed image', image_array[z,:,:])

# Summary of acquisition modelling

We needed
- detection efficiency, attenuation (both are multiplicative effects and incorporated via an `AcquisitionSensitivityModel`)
- randoms and scatter (both are additive effects and incorporated by adding a background term)

Let's see how well the two latter terms fit the measure data. This is easiest to check "outside" the body, i.e. where we did actually not expect any counts at all.

In [ ]:
plt.figure()
plt.plot(acq_data.as_array()[0,z,0,:],'.',markersize=2,label='prompts')
plt.plot(randoms.as_array()[0,z,0,:],'og',markersize=4,label='randoms estimate')
plt.plot((randoms+scatter_estimate).as_array()[0,z,0,:],'xr',markersize=2,label='randoms+scatter estimate')
plt.xlabel('radial distance')
plt.ylabel('counts')
plt.legend()
plt.show()

This doesn't look so good, but of course, there is a lot of noise in the prompt data (and you can see that the detected counts are 0,1,2,...).

So let's sum over all sinograms to reduce noise.

In [ ]:
plt.figure()
plt.plot(numpy.sum(acq_data.as_array()[0,:,0,:],axis=0),'.',markersize=2,label='prompts')
plt.plot(numpy.sum(randoms.as_array()[0,:,0,:],axis=0),'og',markersize=4,label='randoms estimate')
plt.plot(numpy.sum((randoms+scatter_estimate).as_array()[0,:,0,:],axis=0),'xr',markersize=2,label='randoms+scatter estimate')
plt.xlabel('radial distance')
plt.ylabel('counts')
plt.legend()
plt.show()

# What now?
Here are some suggestions for things to try:
- now that you have a good acquisition model, you probably wnat to increase the number of subiterations a bit to get a better quality image.
- change the duration used when going from listmode data to sinograms (longer duration, more counts, less noise), and run at `span=11` and `view_mashing=1` (which is what the mMR does clinically).
- use the final `acq_model` to forward project the reconstructed image. Does it fit the data?
- write a function that takes the listmode data, normalisation file and attenuation image, computes `acq_data` and the acquisition model. Once you have this, you can use your `OSEM` function from a previous notebook (or any other reconstruction method). You can then create a Python script that does everything from start to finish.